# 用户信息获取

In [1]:
# 导入相关的库
import os
import time
import emoji
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from selenium.webdriver.chrome.options import Options

In [2]:
# 首先先进行读取当前路径下的文件
users_df = pd.read_csv('all_users_info_pre.csv')

In [3]:
users_df.head(15)

,user_url,user_name
0,https://www.icourse163.org/home.htm?userId=138...,GreatMarketLeader
1,https://www.icourse163.org/home.htm?userId=115...,杨立宇mooc
2,https://www.icourse163.org/home.htm?userId=103...,席颜
3,https://www.icourse163.org/home.htm?userId=101...,Full_stack
4,https://www.icourse163.org/home.htm?userId=103...,今天美拉学习了吗
5,https://www.icourse163.org/home.htm?userId=111...,aiao1995
6,https://www.icourse163.org/home.htm?userId=101...,顾晨mooc
7,https://www.icourse163.org/home.htm?userId=138...,王海军
8,https://www.icourse163.org/home.htm?userId=114...,Gravemeow
9,https://www.icourse163.org/home.htm?userId=138...,小儿不识月


## 这里是在当前路径下创建一个txt文件
- 如果当前路径下有这个文件了，他就不会再创建了，防止不小心跑到这个代码框的情况

In [4]:
# 创建文件
if not os.path.exists('users_info5.txt'):
    with open('users_info5.txt','a+',encoding='utf-8') as f:
        f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\r".format("user_name","user_url","enrolled_courses_num","follow_num","fans_num","theme_n_reply","thumbs_up","studying_minute","certificate_of_excellence_num","certificate_of_competency_num"))

In [5]:
# 查看txt文件有几行
len(pd.read_table('users_info5.txt')['user_url'].tolist())

0

## 此处可以修改（如果第一次跑or每次都能跑完，就可以自己设定索引直接跑下面的代码）
- 这里50000：70000表示的是第50001条到70000条，也就是$[50001,70001)$左闭右开的情况
- 这里跑5000条大概需要一天10多个小时，可以5000 or 3000看情况跑，比如从50000开始跑3000条只需改成50000：53000即可，跑完修改的就是53000-56000

In [71]:
# 可以进行修改
test_url_lst = users_df.iloc[115000:120000,:]['user_url'].tolist()
test_name_lst = users_df.iloc[115000:120000,:]['user_name'].tolist()

## 如果关了电脑，电脑的内存其实会清空，你不知道它跑到哪里，可以直接跑下面的代码，然后修改最终的区间，就不用跑上面那个手动修改的那个了(第一次跑的话就不要跑下面这两个代码框了，第一次跑必定会报错)

In [86]:
# 首先先进行读取当前路径下的文件pd.read_table('users_info_.txt',lineterminator='\r')
users_df = pd.read_csv('all_users_info_pre.csv')
index_ = users_df['user_url'].tolist().index(pd.read_table('users_info_.txt',lineterminator='\r').iloc[-1:,:]['user_url'].tolist()[0])
index_

117569

In [91]:
test_url_lst = users_df.iloc[(index_+1):120000,:]['user_url'].tolist()
test_name_lst = users_df.iloc[(index_+1):120000,:]['user_name'].tolist()

## 跑用户信息的代码框

In [90]:
# 准备对应的url地址
url = "https://www.icourse163.org/"

# 创建一个参数对象，用来控制chrome以无界面模式打开
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')

# 首先创建一个driver实例对象
driver = webdriver.Chrome(chrome_options=chrome_options)
driver.maximize_window()

# 避开 chrome 检测
driver.execute_cdp_cmd(
    "Page.addScriptToEvaluateOnNewDocument",
    {
        "source": """Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"""
    }
)
driver.implicitly_wait(5)


for url,name in zip(test_url_lst,test_name_lst):
    '''
    参加的课程数量，关注人数，粉丝数，主题/回复，获赞数量，学习时长，优秀证书数量，合格证书数量,url,name
    '''
    # url = 'https://www.icourse163.org/home.htm?userId=1015300339'
    # name = '顾晨mooc'
    # 在新窗口打开
    driver.execute_script("window.open('{}');".format(url))

    # 切换窗口
    handles = driver.window_handles
    driver.switch_to.window(handles[-1])

    time.sleep(1)
    # 首先进行判断他是学生还是教授
    bollen_damn_lst = [damn1.text for damn1 in driver.find_elements(by=By.CSS_SELECTOR, value="#j-module-tab > div > div > a > span")]
    #print(bollen_damn_lst)
    if len(bollen_damn_lst) == 1:
        # 这个表示的是学生
        # 1. 提取出参加的课程数量
        enrolled_courses_num = bollen_damn_lst[0].split('课程')[1]
        if len(enrolled_courses_num) == 0:
            enrolled_courses_num = '0'

        # 2. 提取出关注的人数
        follow_num = driver.find_element(by=By.CSS_SELECTOR, value="#j-self-content > div > div.u-ui-f2f > span > span:nth-child(1)").text[2:-1]

        # 3. 提取出对应的粉丝数
        fans_num = driver.find_element(by=By.CSS_SELECTOR, value="#j-self-content > div > div.u-ui-f2f > span > span:nth-child(2)").text[2:-1]

        # 4. 主题/回复
        theme_n_reply = driver.find_element(by=By.CSS_SELECTOR, value="#j-self-content > div > div.u-ui-box.f-fr > div > div > div.u-ui-discuss-cont > span").text

        # 5. 学习时长
        studying_duration = driver.find_element(by=By.CSS_SELECTOR, value="#j-self-content > div > div.u-ui-box.f-fr > div > div > div.u-ui-time-cont > span").text
        if "时" in studying_duration:
            ch1 = studying_duration.index("时")
            # 转换成分钟
            studying_minute = int(studying_duration[:ch1])*60 + int(studying_duration[ch1+1:-1])
        else:
            studying_minute = studying_duration[:-1]

        # 获赞数量
        thumbs_up = driver.find_element(by=By.CSS_SELECTOR,value="#j-self-content > div > div.u-ui-box.f-fr > div > div > div.u-ui-zan-cnt > span").text

        # /html/body/div[4]/div[2]/div[3]/div/div[1]/div[3]/div/div/div[3]/div/ul/li
        # /html/body/div[4]/div[2]/div[3]/div/div[1]/div[3]/div/div/div[3]/div/ul/li
        driver.get("{}{}".format(url,"#/home/mycert"))
        driver.find_element(by=By.XPATH,value='/html/body/div[4]/div[2]/div[3]/div/div[1]/div[3]/div/div/div[1]/div/div[2]/span').click()
        # 首先为了提高效率，我们将会先进行判断全部的证书
        all_certs_lst = driver.find_elements(by=By.XPATH,value='/html/body/div[4]/div[2]/div[3]/div/div[1]/div[3]/div/div/div[2]/div')
        if len(all_certs_lst) == 0:
            perfect_certs_num = 0
            pass_certs_num = 0
        else:
            # 点击优秀证书
            time.sleep(0.5)
            driver.find_element(by=By.XPATH,value='/html/body/div[4]/div[2]/div[3]/div/div[1]/div[3]/div/div/div[1]/div/div[2]/span').click()
            time.sleep(1)
            # 查看是否有换页button
            change_pages_button = driver.find_elements(by=By.XPATH,value='/html/body/div[4]/div[2]/div[3]/div/div[1]/div[3]/div/div/div[3]/div/ul/li')
            if len(change_pages_button) == 0:
                # 这里表示的是只有一页
                perfect_certs_num = len(driver.find_elements(by=By.CSS_SELECTOR, value="div.u-certCard-courseFunc_card_title.f-thide"))
            else:
                # 这里表示的是不止一页
                tp_page_num = len(change_pages_button) - 2
                # 点击最后一页
                change_pages_button[-2].click()
                time.sleep(0.7)
                perfect_certs_num = len(driver.find_elements(by=By.CSS_SELECTOR, value="div.u-certCard-courseFunc_card_title.f-thide")) + (tp_page_num-1)*40
                #print(perfect_certs_num)

            # 点击合格证书
            driver.find_element(by=By.XPATH,value='/html/body/div[4]/div[2]/div[3]/div/div[1]/div[3]/div/div/div[1]/div/div[3]/span').click()
            time.sleep(0.7)
            # 查看是否有换页button
            change_pages_button2 = driver.find_elements(by=By.XPATH,value='/html/body/div[4]/div[2]/div[3]/div/div[1]/div[3]/div/div/div[3]/div/ul/li')
            if len(change_pages_button2) == 0:
                # 这里表示的是只有一页
                pass_certs_num = len(driver.find_elements(by=By.CSS_SELECTOR, value="div.u-certCard-courseFunc_card_title.f-thide"))
            else:
                # 这里表示的是不止一页
                tp_page_num = len(change_pages_button2) - 2
                # 点击最后一页
                change_pages_button2[-2].click()
                time.sleep(1)
                pass_certs_num = len(driver.find_elements(by=By.CSS_SELECTOR, value="div.u-certCard-courseFunc_card_title.f-thide")) + (tp_page_num-1)*40
                #print(pass_certs_num)
        driver.close()
        # 切换窗口
        handles = driver.window_handles
        driver.switch_to.window(handles[-1])
        #print('-----------------------------')
        want_to_write = "{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\r".format(name,url,enrolled_courses_num,follow_num,fans_num,theme_n_reply,thumbs_up,studying_minute,perfect_certs_num,pass_certs_num)

        with open('users_info_.txt','a+',encoding='utf-8') as f:
            f.write(want_to_write)
    else:
        driver.close()
        # 切换窗口
        handles = driver.window_handles
        driver.switch_to.window(handles[-1])

/var/folders/6b/xp2m5pws089bz4bpfh9zpndc0000gn/T/ipykernel_67240/4033381788.py:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 103
Current browser version is 105.0.5195.102 with binary path /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
Stacktrace:
0   chromedriver                        0x000000010455dd14 chromedriver + 3792148
1   chromedriver                        0x00000001044f4828 chromedriver + 3360808
2   chromedriver                        0x00000001041f50b8 chromedriver + 217272
3   chromedriver                        0x0000000104217ce0 chromedriver + 359648
4   chromedriver                        0x00000001042146b4 chromedriver + 345780
5   chromedriver                        0x000000010421168c chromedriver + 333452
6   chromedriver                        0x0000000104242074 chromedriver + 532596
7   chromedriver                        0x000000010421a318 chromedriver + 369432
8   chromedriver                        0x00000001045331e8 chromedriver + 3617256
9   chromedriver

 ## debug代码框
- 两种需要跑下面代码框的情况
    - 如果上面的代码框报错，需要跑下面第一个代码框，然后重新跑上面的代码框（爬取用户信息的代码框即可）
    - 如果想要停止上面的代码框一下，可以按上面的红色正方形按钮，然后不关pycharm的话，停止后可以跑这个代码框再跑上面的代码框

In [81]:
# debug
new_index = test_url_lst.index(url)
print(new_index)
test_url_lst = test_url_lst[new_index:]
test_name_lst = test_name_lst[new_index:]

ValueError: 'https://www.icourse163.org/' is not in list

## 查看代码框

In [15]:
pd.read_table('users_info_.txt')

ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.


## users_info_.txt为90000以上的数据

In [82]:
import pandas as pd
pd.read_table('users_info_.txt',lineterminator='\r')

,user_name,user_url,enrolled_courses_num,follow_num,fans_num,theme_n_reply,thumbs_up,studying_minute,certificate_of_excellence_num,certificate_of_competency_num
0,mooc78138892619785614,https://www.icourse163.org/home.htm?userId=114...,97,0.0,0.0,0.0,4.0,34886.0,0,0
1,宋天都,https://www.icourse163.org/home.htm?userId=138...,47,0.0,0.0,96.0,21.0,7713.0,1,11
2,星云丶丶丶,https://www.icourse163.org/home.htm?userId=102...,81,0.0,0.0,0.0,0.0,496.0,0,0
3,河蟹君,https://www.icourse163.org/home.htm?userId=120...,1,0.0,0.0,0.0,0.0,367.0,0,1
4,mooc17365938131719756,https://www.icourse163.org/home.htm?userId=138...,7,0.0,0.0,0.0,0.0,2679.0,0,1
...,...,...,...,...,...,...,...,...,...,...
24848,mooc77561791391688094,https://www.icourse163.org/home.htm?userId=138...,3,0.0,0.0,0.0,0.0,1589.0,0,0
24849,罗斯开了二倍速,https://www.icourse163.org/home.htm?userId=114...,21,2.0,0.0,0.0,0.0,1048.0,0,0
24850,爱学Python的铁头娃,https://www.icourse163.org/home.htm?userId=138...,14,5.0,0.0,15.0,9.0,7971.0,0,0
24851,同学ykt1498269888684,https://www.icourse163.org/home.htm?userId=102...,137,2.0,0.0,2.0,0.0,13168.0,0,0
